In [45]:
import numpy as np
import imageio
from PIL import Image

### Wczytanie zdjęcia

In [46]:
image = imageio.imread('Zdjecia/SADDLE_PE.JPG')

### Rozmiar zdjęcia

In [47]:
image.shape

(876, 1053, 3)

### Zapisanie wysokości i szerokości zdjęcia

In [26]:
n_x = len(image)
n_y = len(image[0])

### Utworzenia nowej tablicy (image_m) o wymiarach n_x i n_y, ale będzie ona przechowywała tylko jedną wartość (ponieważ dla odcień szarość R=G=B)

In [48]:
image_m = [ [ None for y in range( n_y ) ] for x in range( n_x ) ]

for x in range(n_x):
    for y in range(n_y):
        image_m[x][y] = image[x][y][0]

### Rozmiar tablicy

In [50]:
np.shape(image_m)

(876, 1053)